# 使用SageMaker Training Jobs对对话摘要的Instructor-Mode进行微调


<a name='1'></a>
## Set up Kernel and Required Dependencies

First, check that the correct kernel is chosen.

<img src="img/kernel_set_up.png" width="300"/>

You can click on that to see and check the details of the image, kernel, and instance type.

<img src="img/w3_kernel_and_instance_type.png" width="600"/>

# NOTE:  THIS NOTEBOOK WILL TAKE ABOUT 20 MINUTES TO COMPLETE.

# PLEASE BE PATIENT.

In [2]:
# 导入必要的库
import boto3
import sagemaker
import pandas as pd

# 创建一个 SageMaker 会话
sess = sagemaker.Session()
# 获取默认的 S3 存储桶
bucket = sess.default_bucket()
# 获取执行角色。这个角色是 AWS SageMaker 用来访问 AWS 资源的。
role = sagemaker.get_execution_role()
# 获取当前的 AWS 区域名称
region = boto3.Session().region_name

# 导入 botocore.config，它是 AWS SDK for Python (Boto3) 的一部分，
# 用于提供低级别的、核心的、与 AWS 服务交互的功能
import botocore.config

# 创建一个 botocore 配置对象，设置 user_agent_extra 参数为 'dsoaws/2.0'。
# user_agent_extra 是一个可以添加到 user-agent header 中的字符串，
# 它可以帮助追踪和识别通过 SDK 发送的请求。
config = botocore.config.Config(
    user_agent_extra='dsoaws/2.0'
)

# 创建一个 boto3 客户端来访问 SageMaker 服务。
# 这个客户端将使用之前创建的 AWS 区域名称和 botocore 配置对象。
sm = boto3.Session().client(service_name="sagemaker", 
                            region_name=region, 
                            config=config)

# _前提条件：在继续使用此笔记本之前，你需要成功运行PREPARE部分的笔记本_

In [3]:
%store -r processed_train_data_s3_uri

In [4]:
try:
    processed_train_data_s3_uri
except NameError:
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] Please run the notebooks in the PREPARE section before you continue.")
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")

In [5]:
print(processed_train_data_s3_uri)

s3://sagemaker-us-east-1-941797585610/sagemaker-scikit-learn-2023-09-08-16-00-28-289/output/train


In [6]:
%store -r processed_validation_data_s3_uri

In [7]:
try:
    processed_validation_data_s3_uri
except NameError:
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] Please run the notebooks in the PREPARE section before you continue.")
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")

In [8]:
print(processed_validation_data_s3_uri)

s3://sagemaker-us-east-1-941797585610/sagemaker-scikit-learn-2023-09-08-16-00-28-289/output/validation


In [9]:
%store -r processed_test_data_s3_uri

In [10]:
try:
    processed_test_data_s3_uri
except NameError:
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] Please run the notebooks in the PREPARE section before you continue.")
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")

In [11]:
print(processed_test_data_s3_uri)

s3://sagemaker-us-east-1-941797585610/sagemaker-scikit-learn-2023-09-08-16-00-28-289/output/test


In [12]:
%store -r model_checkpoint

In [13]:
try:
    model_checkpoint
except NameError:
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] Please run the notebooks in the PREPARE section before you continue.")
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")

In [14]:
print(model_checkpoint)

google/flan-t5-base


# 指定S3中的数据集
我们正在使用在上一节中创建的训练、验证和测试数据集划分

In [15]:
print(processed_train_data_s3_uri)

!aws s3 ls $processed_train_data_s3_uri/

s3://sagemaker-us-east-1-941797585610/sagemaker-scikit-learn-2023-09-08-16-00-28-289/output/train
2023-09-08 16:07:05    2540571 1694189218128.parquet
2023-09-08 16:07:04    2545157 1694189219320.parquet


In [16]:
print(processed_validation_data_s3_uri)

!aws s3 ls $processed_validation_data_s3_uri/

s3://sagemaker-us-east-1-941797585610/sagemaker-scikit-learn-2023-09-08-16-00-28-289/output/validation
2023-09-08 16:07:05     150701 1694189218128.parquet
2023-09-08 16:07:04     150220 1694189219320.parquet


In [17]:
print(processed_test_data_s3_uri)

!aws s3 ls $processed_test_data_s3_uri/

s3://sagemaker-us-east-1-941797585610/sagemaker-scikit-learn-2023-09-08-16-00-28-289/output/test
2023-09-08 16:07:06     157115 1694189218128.parquet
2023-09-08 16:07:05     153865 1694189219320.parquet


# 指定S3输入数据

In [18]:
# 导入 sagemaker.inputs 下的 TrainingInput 类
from sagemaker.inputs import TrainingInput

# 使用处理后的训练数据的 S3 URI 创建一个 TrainingInput 对象
s3_input_train_data = TrainingInput(s3_data=processed_train_data_s3_uri)
# 使用处理后的验证数据的 S3 URI 创建一个 TrainingInput 对象
s3_input_validation_data = TrainingInput(s3_data=processed_validation_data_s3_uri)
# 使用处理后的测试数据的 S3 URI 创建一个 TrainingInput 对象
s3_input_test_data = TrainingInput(s3_data=processed_test_data_s3_uri)

# 打印训练数据的 TrainingInput 对象的配置信息
print(s3_input_train_data.config)
# 打印验证数据的 TrainingInput 对象的配置信息
print(s3_input_validation_data.config)
# 打印测试数据的 TrainingInput 对象的配置信息
print(s3_input_test_data.config)

{'DataSource': {'S3DataSource': {'S3DataType': 'S3Prefix', 'S3Uri': 's3://sagemaker-us-east-1-941797585610/sagemaker-scikit-learn-2023-09-08-16-00-28-289/output/train', 'S3DataDistributionType': 'FullyReplicated'}}}
{'DataSource': {'S3DataSource': {'S3DataType': 'S3Prefix', 'S3Uri': 's3://sagemaker-us-east-1-941797585610/sagemaker-scikit-learn-2023-09-08-16-00-28-289/output/validation', 'S3DataDistributionType': 'FullyReplicated'}}}
{'DataSource': {'S3DataSource': {'S3DataType': 'S3Prefix', 'S3Uri': 's3://sagemaker-us-east-1-941797585610/sagemaker-scikit-learn-2023-09-08-16-00-28-289/output/test', 'S3DataDistributionType': 'FullyReplicated'}}}


# 为FLAN模型设置超参数

In [19]:
# 设置训练的轮数。如果希望训练更长时间，可以增加这个值
epochs = 1 
# 设置学习率
learning_rate = 0.00001
# 设置权重衰减系数
weight_decay = 0.01
# 设置训练批次的大小
train_batch_size = 4
# 设置验证批次的大小
validation_batch_size = 4
# 设置测试批次的大小
test_batch_size = 4
# 设置训练实例的数量
train_instance_count = 1
# 设置训练实例的类型
train_instance_type = "ml.c5.9xlarge"
# 设置训练实例的磁盘空间大小（单位：GB）
train_volume_size = 1024
# 设置输入模式为 "FastFile"，这是一个 SageMaker 提供的高效的输入模式
# FastFile 是 AWS SageMaker 中的一种数据输入模式。在 SageMaker 中，训练模型时，可以选择三种输入模式之一： File、 Pipe 和 FastFile。
# File 模式：在训练开始之前，会将所有训练数据下载到训练实例上。这种模式的优点是训练速度快，因为所有数据都在本地。缺点是需要更多的存储空间，并且在训练开始之前需要花费时间下载数据。
# Pipe 模式：这是一种流式传输模式，数据在需要时才从 S3 传输到训练实例。这种模式的优点是不需要额外的存储空间，并且不需要在训练开始之前下载所有数据。缺点是训练速度可能会受到数据传输速度的影响。
# FastFile 模式：这是一种介于 File 和 Pipe 模式之间的新模式。在 FastFile 模式下，数据会以流的形式传输到训练实例，并在传输过程中存储到本地。这种模式结合了 File 模式和 Pipe 模式的优点，可以在不牺牲训练速度的情况下，减少存储空间的需求和减少数据下载的时间。
input_mode = "FastFile"
# 设置训练样本的百分比。如果希望在更多的数据上进行训练，可以增加这个值
train_sample_percentage = 0.01

# 设置模型性能的指标

In [22]:
# 定义了一个 metrics_definitions 列表，该列表包含两个字典，用于指定 SageMaker 应如何解析训练日志以获取训练和验证损失。
# "Name" 是指标的名称，"Regex" 是一个用于从训练日志中提取指标值的正则表达式。
# 在运行训练作业时，SageMaker 会使用这些正则表达式来解析训练日志，并将解析出的指标值发送到 CloudWatch Metrics。
# 这样就可以在 CloudWatch 中查看这些指标，以便监控训练过程。

# 定义 metrics_definitions 列表
metrics_definitions = [
    # 指定 "train:loss" 的正则表达式用于从训练日志中提取训练损失
    {"Name": "train:loss", "Regex": "'train_loss': ([0-9\\.]+)"},
    # 指定 "validation:loss" 的正则表达式用于从训练日志中提取验证损失
    {"Name": "validation:loss", "Regex": "'eval_loss': ([0-9\\.]+)"},
]

# 指定S3 Checkpoint位置

用于Spot实例训练。如果节点被替换，新节点将从最新的检查点开始训练。

In [23]:
# 导入 uuid 库
import uuid

# 生成一个唯一的检查点 S3 前缀
checkpoint_s3_prefix = "checkpoints/{}".format(str(uuid.uuid4()))
# 使用默认的 S3 存储桶和生成的检查点 S3 前缀创建一个完整的检查点 S3 URI
checkpoint_s3_uri = "s3://{}/{}/".format(bucket, checkpoint_s3_prefix)

# 打印出检查点 S3 URI
print(checkpoint_s3_uri)

s3://sagemaker-us-east-1-941797585610/checkpoints/f49a5ed4-9e0a-4248-863c-247f5c386696/


# 设置在SageMaker上运行的脚本
准备在SageMaker上运行的模型

In [ ]:
# 'pygmentize' 是 Pygments 包的一个命令行工具，用于对源代码进行语法高亮
# 'src/train.py' 是想要高亮显示的 Python 源代码文件的路径
# '!' 符号是在 Jupyter notebook 中运行系统shell命令的方式
# 这个命令的作用是在 Jupyter notebook 中高亮显示 'src/train.py' 文件的源代码
!pygmentize src/train.py

In [24]:
# 从 sagemaker.pytorch 包中导入 PyTorch 类
from sagemaker.pytorch import PyTorch

# 创建 PyTorch estimator，这是一个 SageMaker 的训练作业的抽象
# 用于设置训练作业的参数，并启动和管理训练作业
estimator = PyTorch(
    # 'entry_point' 参数指定了训练脚本的路径
    entry_point="train.py",  
    # 'source_dir' 参数指定了包含训练脚本和其他辅助脚本的目录的路径
    source_dir="src",
    # 'role' 参数指定了执行训练作业的 AWS IAM 角色
    role=role,
    # 'instance_count' 参数指定了训练作业要使用的 Amazon EC2 实例的数量
    instance_count=train_instance_count,
    # 'instance_type' 参数指定了训练作业要使用的 Amazon EC2 实例的类型
    instance_type=train_instance_type,
    # 'volume_size' 参数指定了附加到每个训练实例的 Amazon EBS 卷的大小（单位：GB）
    volume_size=train_volume_size,
    # 'checkpoint_s3_uri' 参数指定了保存模型检查点的 Amazon S3 路径
    checkpoint_s3_uri=checkpoint_s3_uri,
    # 'py_version' 参数指定了要使用的 Python 版本
    py_version="py39",
    # 'framework_version' 参数指定了要使用的 PyTorch 版本
    framework_version="1.13",
    # 'hyperparameters' 参数是一个字典，定义了训练作业中模型的超参数
    hyperparameters={
        "epochs": epochs,
        "learning_rate": learning_rate,
        "weight_decay": weight_decay,        
        "train_batch_size": train_batch_size,
        "validation_batch_size": validation_batch_size,
        "test_batch_size": test_batch_size,
        "model_checkpoint": model_checkpoint,
        "train_sample_percentage": train_sample_percentage,
    },
    # 'input_mode' 参数指定了训练数据的输入模式
    input_mode=input_mode,
    # 'metric_definitions' 参数是一个列表，定义了训练作业的度量标准
    metric_definitions=metrics_definitions,
)

# 在 SageMaker 上训练模型

In [25]:
# 使用 'estimator.fit' 方法启动训练作业
# 'inputs' 参数是一个字典，指定了训练、验证和测试数据的 S3 路径
# 'wait' 参数设置为 'False'，意味着在训练作业启动后，该方法将立即返回，而不会等待训练作业完成
estimator.fit(
    # 在 AWS SageMaker 的训练作业中，inputs 字典的键（key）不是固定的。你可以根据需要自定义这些键。在上面的代码中，inputs 字典的键为 "train"、"validation" 和 "test"，这只是一种常见的设定，用于区分训练、验证和测试数据。
    # 这些键对应的值应该是 Amazon S3 的 URIs，表示数据的存储位置。在训练作业中，你可以通过这些键来访问对应的数据。
    # 例如，假设你在训练脚本中有以下代码：
    # def train(channel_input_dirs):
    #    train_data_dir = channel_input_dirs['train']
    #    validation_data_dir = channel_input_dirs['validation']
    #    test_data_dir = channel_input_dirs['test']
    # ...
    # 在这个例子中，channel_input_dirs 字典的键就对应 estimator.fit 方法中 inputs 字典的键。所以，你可以用这些键来获取对应的数据。
    inputs={"train": s3_input_train_data, "validation": s3_input_validation_data, "test": s3_input_test_data},
    wait=False,
)

INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: pytorch-training-2023-09-09-04-00-02-055


In [26]:
# 'estimator.latest_training_job.name' 获取最新的训练作业的名称
# 'latest_training_job' 是 'estimator' 对象的一个属性，表示最新启动的训练作业
# 'name' 是 'latest_training_job' 的一个属性，表示训练作业的名称
training_job_name = estimator.latest_training_job.name

# 使用 'print' 函数打印训练作业的名称
print("Training Job Name: {}".format(training_job_name))

Training Job Name: pytorch-training-2023-09-09-04-00-02-055


In [ ]:
from IPython.core.display import display, HTML

display(
    HTML(
        '<b>Review <a target="blank" href="https://console.aws.amazon.com/sagemaker/home?region={}#/jobs/{}">Training Job</a> After About 5 Minutes</b>'.format(
            region, training_job_name
        )
    )
)

In [ ]:
from IPython.core.display import display, HTML

display(
    HTML(
        '<b>Review <a target="blank" href="https://console.aws.amazon.com/cloudwatch/home?region={}#logStream:group=/aws/sagemaker/TrainingJobs;prefix={};streamFilter=typeLogStreamPrefix">CloudWatch Logs</a> After About 5 Minutes</b>'.format(
            region, training_job_name
        )
    )
)

In [ ]:
from IPython.core.display import display, HTML

display(
    HTML(
        '<b>Review <a target="blank" href="https://s3.console.aws.amazon.com/s3/buckets/{}/{}/?region={}&tab=overview">S3 Output Data</a> After The Training Job Has Completed</b>'.format(
            bucket, training_job_name, region
        )
    )
)

In [ ]:
%%time

estimator.latest_training_job.wait(logs=False)

# 将经过微调的模型部署到实时endpoint

In [28]:
# 使用 'create_model' 方法创建一个 SageMaker 模型
# 'entry_point' 参数指定了推理脚本的路径
# 'source_dir' 参数指定了包含推理脚本和其他辅助脚本的目录的路径
# 'create_model' 方法根据最新的训练作业创建一个模型，这个模型可以用于部署推理端点
sm_model = estimator.create_model(
    entry_point='inference.py',
    source_dir='src',
)

# 使用 'replace' 方法将训练作业名称中的 'pytorch-training' 替换为 'summary-tuned'
# 这是为了创建一个新的端点名称
endpoint_name = training_job_name.replace('pytorch-training', 'summary-tuned')

# 使用 'deploy' 方法部署一个推理端点
# 'initial_instance_count' 参数指定了端点要使用的 Amazon EC2 实例的数量
# 'instance_type' 参数指定了端点要使用的 Amazon EC2 实例的类型
# 'endpoint_name' 参数指定了新的端点的名称
# 'deploy' 方法将在 Amazon SageMaker 服务上创建一个新的推理端点，并返回一个预测器对象
# 这个预测器对象可以用于向新的推理端点发送推理请求
predictor = sm_model.deploy(
    initial_instance_count=1,
    instance_type='ml.m5.2xlarge',
    endpoint_name=endpoint_name
)

INFO:sagemaker:Creating model with name: pytorch-training-2023-09-09-04-45-03-835
INFO:sagemaker:Creating endpoint-config with name summary-tuned-2023-09-09-04-00-02-055
INFO:sagemaker:Creating endpoint with name summary-tuned-2023-09-09-04-00-02-055


-----!

# 在 SageMaker 端点中使用微调模型进行Zero Shot Inference

In [29]:
zero_shot_prompt = """Summarize the following conversation.

#Person1#: Tom, I've got good news for you.
#Person2#: What is it?
#Person1#: Haven't you heard that your novel has won The Nobel Prize?
#Person2#: Really? I can't believe it. It's like a dream come true. I never expected that I would win The Nobel Prize!
#Person1#: You did a good job. I'm extremely proud of you.
#Person2#: Thanks for the compliment.
#Person1#: You certainly deserve it. Let's celebrate!

Summary:"""

In [32]:
# 导入 'json' 模块，用于处理 JSON 数据
import json

# 从 'sagemaker' 包中导入 'Predictor' 类
# 'Predictor' 类是用于向 SageMaker 推理端点发送数据并接收预测结果的类
from sagemaker import Predictor

# 创建一个 'Predictor' 对象
# 'endpoint_name' 参数指定了推理端点的名称
# 'sagemaker_session' 参数指定了 SageMaker 会话，这个会话包含了与 AWS 服务的连接和配置
predictor = Predictor(
    endpoint_name=endpoint_name,
    sagemaker_session=sess,
)

# 使用 'predict' 方法向推理端点发送数据，并接收预测结果
# 第一个参数是要预测的数据
# 第二个参数是一个字典，指定了请求的内容类型和接收的响应类型
response = predictor.predict(zero_shot_prompt,
        {
            "ContentType": "application/x-text",
            "Accept": "application/json",
        },
)

# 使用 'json.loads' 方法将响应的 JSON 数据解码为 Python 对象
# 'response.decode('utf-8')' 将响应的二进制数据解码为 UTF-8 编码的字符串
response_json = json.loads(response.decode('utf-8'))

# 使用 'print' 函数打印解码后的响应数据
print(response_json)

Tom's novel has won the Nobel Prize.


# 删除端点

In [33]:
# predictor.delete_endpoint()

INFO:sagemaker:Deleting endpoint configuration with name: summary-tuned-2023-09-09-04-00-02-055
INFO:sagemaker:Deleting endpoint with name: summary-tuned-2023-09-09-04-00-02-055


# Release Resources

In [ ]:
%%html

<p><b>Shutting down your kernel for this notebook to release resources.</b></p>
<button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernel</button>
        
<script>
try {
    els = document.getElementsByClassName("sm-command-button");
    els[0].click();
}
catch(err) {
    // NoOp
}    
</script>